### Tokenize:


In [ ]:
# # if needed:
# install dependencies
# !pip install torch
# !pip install transformers[torch]
# !pip install accelerate

In [ ]:
# imports
import torch
import pandas as pd
import numpy as np
from torch.utils.data.dataloader import DataLoader
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import random_split, DataLoader
import gc
import math

np.random.seed(123)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # if using GPU

In [ ]:
# mount drive
drive.mount('/content/drive')

# define path to folder - CHANGE TO WHICHEVER DIRECTORY YOU WANT
path_folder_in = f"/content/drive/MyDrive/Deep_Learning_project/chunks"
path_folder_out = f"/content/drive/MyDrive/Deep_Learning_project/splits"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load pre-trained CausalLM GPT-2 model and the matching tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2", bos_token='<sos>', eos_token='<eos>', pad_token='<pad>')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# add special tokens again, ensure they are not already added
if '[Genre: Pop]' not in tokenizer.get_vocab():
    tokenizer.add_special_tokens({'additional_special_tokens': ['[Genre: Pop]']})
if '[Genre: Rock]' not in tokenizer.get_vocab():
    tokenizer.add_special_tokens({'additional_special_tokens': ['[Genre: Rap]']})

# check the tokens were added
print('[Genre: Pop]' in tokenizer.get_vocab())
print('[Genre: Rap]' in tokenizer.get_vocab())

True
True


In [ ]:
# define your preprocessing function
def preprocess_csv(filename, tokenizer):
  df = pd.read_csv(filename)
  df = df.dropna()

  # filter out lyrics with more than 400 words and less then 20 words
  df = df[df['lyric'].apply(lambda x: len(tokenizer.encode(x)) < 400)]
  df = df[df['lyric'].apply(lambda x: len(tokenizer.encode(x)) > 20)]

  # downsample pop chunk to size of rap chunk
  rap_count = (df['class'] == 0).sum()
  pop_count = (df['class'] == 1).sum()

  if pop_count > rap_count:
      pop_indices = df[df['class'] == 1].index
      drop_indices = np.random.choice(pop_indices, size=(pop_count - rap_count), replace=False) # randomly select indices to *drop* to match the number of rap songs
      df = df.drop(drop_indices)   # drop the selected pop songs- now the number of pop and rap songs are equal
      # print("Pop chunk downsampled to match rap chunk:\n", df['class'].value_counts())

  # convert lyrics to list
  songs = df["lyric"].tolist()

  # initialize lists to store input ids and attention masks
  input_ids = []
  attention_masks = []

  # iterate over every song and encode it using the tokenizer
  for song in songs:
      encodings = tokenizer('<sos>' + song + '<eos>', truncation=True, max_length= 400, padding="max_length")
      input_ids.append(torch.tensor(encodings['input_ids']))
      attention_masks.append(torch.tensor(encodings['attention_mask']))

  return input_ids, attention_masks

In [ ]:
num_chunks = 40

# list to store results from each file
all_input_ids = []
all_attention_masks = []

# loop over each CSV file
for i in range(1, num_chunks+1):  # we have 20 CSV files
    # construct filename
    filename = f"{path_folder_in}/chunk_{i}.csv"
    print(i)
    # preprocess CSV file
    input_ids, attention_masks = preprocess_csv(filename, tokenizer)

    # append results to the lists
    all_input_ids.extend(input_ids)
    all_attention_masks.extend(attention_masks)



1


Token indices sequence length is longer than the specified maximum sequence length for this model (1267 > 1024). Running this sequence through the model will result in indexing errors


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


In [ ]:
# concatenate the lists into tensors and save the files
np.save(f"{path_folder_out}/all_input_ids.npy", np.array(all_input_ids))
np.save(f"{path_folder_out}/all_attention_masks.npy", np.array(all_attention_masks))

In [ ]:
tokenizer.save_pretrained(f'{path_folder_out}')

('/content/drive/MyDrive/Deep_Learning_project/splits/tokenizer_config.json',
 '/content/drive/MyDrive/Deep_Learning_project/splits/special_tokens_map.json',
 '/content/drive/MyDrive/Deep_Learning_project/splits/vocab.json',
 '/content/drive/MyDrive/Deep_Learning_project/splits/merges.txt',
 '/content/drive/MyDrive/Deep_Learning_project/splits/added_tokens.json',
 '/content/drive/MyDrive/Deep_Learning_project/splits/tokenizer.json')